In [10]:
import os
import pandas as pd
import pm4py

Data Loading:

In [28]:
file_dir = '../data/'
file_name = 'Hospital_Billing_Eventlog.xes'
download_url = 'https://data.4tu.nl/file/6af6d5f0-f44c-49be-aac8-8eaa5fe4f6fd/28b83e72-375e-4da4-8459-a8506e898edf'
# Check if file exists and download it if not

# Check if directory exists
if not os.path.exists(file_dir):
    os.system("mkdir " + file_dir)
if not os.path.exists(file_dir + file_name):
    print('Downloading file...')
    os.system(f'wget -O {file_dir + file_name + ".gz"} {download_url}')
    os.system(f'gzip -d {file_dir + file_name + ".gz"}')


In [29]:
data = pm4py.read_xes(file_dir + file_name)
data = pm4py.format_dataframe(data)

parsing log, completed traces :: 100%|██████████| 100000/100000 [00:11<00:00, 8416.25it/s]


In [30]:
# Get overview over data
data

,isCancelled,diagnosis,time:timestamp,caseType,speciality,org:resource,concept:name,blocked,isClosed,flagD,...,closeCode,actRed,actOrange,flagC,msgCount,version,msgType,msgCode,@@index,@@case_index
0,False,A,2012-12-16 19:33:10+00:00,A,A,ResA,NEW,False,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,NaN,NaN,2013-12-15 19:00:37+00:00,NaN,NaN,NaN,FIN,NaN,NaN,NaN,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2,NaN,NaN,2013-12-16 03:53:38+00:00,NaN,NaN,NaN,RELEASE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
3,NaN,NaN,2013-12-17 12:56:29+00:00,NaN,NaN,NaN,CODE OK,NaN,NaN,NaN,...,NaN,False,False,False,0.0,A,NaN,NaN,3,0
4,NaN,NaN,2013-12-19 03:44:31+00:00,NaN,NaN,ResB,BILLED,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451354,False,UE,2015-06-27 02:01:17+00:00,A,H,ResA,NEW,False,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,451354,99999
451355,NaN,NaN,2015-07-06 23:57:45+00:00,NaN,NaN,NaN,FIN,NaN,NaN,NaN,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,451355,99999
451356,NaN,NaN,2015-07-15 05:39:38+00:00,NaN,NaN,NaN,RELEASE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,451356,99999
451357,NaN,NaN,2015-07-15 06:33:01+00:00,NaN,NaN,NaN,CODE OK,NaN,NaN,NaN,...,NaN,False,False,False,0.0,E,NaN,NaN,451357,99999


Preprocessing:

In [34]:
# print columns
data = data[['@@case_index', 'diagnosis', 'concept:name', 'isCancelled', 'isClosed']]
data

,@@case_index,diagnosis,concept:name,isCancelled,isClosed
0,0,A,NEW,False,True
1,0,NaN,FIN,NaN,NaN
2,0,NaN,RELEASE,NaN,NaN
3,0,NaN,CODE OK,NaN,NaN
4,0,NaN,BILLED,NaN,NaN
...,...,...,...,...,...
451354,99999,UE,NEW,False,True
451355,99999,NaN,FIN,NaN,NaN
451356,99999,NaN,RELEASE,NaN,NaN
451357,99999,NaN,CODE OK,NaN,NaN


In [35]:
# Save as new, smaller csv
data.to_csv('../data/preprocessed_data.csv', sep=';', index=False)

Feature Extraction:

In [36]:
# Overview over unique values for each column
print("Unique value oveview:\n")
print("======================")
print(data.nunique())
print("======================")

Unique value oveview:

@@case_index    100000
diagnosis         1192
concept:name        18
isCancelled          2
isClosed             2
dtype: int64


In [37]:
# Get traces
traces = data.groupby('@@case_index')['concept:name'].apply(tuple).reset_index().rename(columns={"concept:name": "Traces"})

In [38]:
traces

,@@case_index,Traces
0,0,"(NEW, FIN, RELEASE, CODE OK, BILLED)"
1,1,"(NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL..."
2,2,"(NEW,)"
3,3,"(NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL..."
4,4,"(NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL..."
...,...,...
99995,99995,"(NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL..."
99996,99996,"(NEW, DELETE)"
99997,99997,"(NEW, CHANGE DIAGN, DELETE)"
99998,99998,"(NEW,)"


In [39]:
# Get the value counts for each unique trace
trace_counts = traces['Traces'].value_counts()

# Convert to dictionary
trace_counts_dict = trace_counts.to_dict()

In [40]:
trace_counts_dict

{('NEW', 'CHANGE DIAGN', 'FIN', 'RELEASE', 'CODE OK', 'BILLED'): 33672,
 ('NEW',): 22373,
 ('NEW', 'FIN', 'RELEASE', 'CODE OK', 'BILLED'): 20901,
 ('NEW', 'DELETE'): 4813,
 ('NEW', 'FIN'): 3508,
 ('NEW',
  'CHANGE DIAGN',
  'CHANGE DIAGN',
  'FIN',
  'RELEASE',
  'CODE OK',
  'BILLED'): 2121,
 ('NEW', 'CHANGE DIAGN', 'DELETE'): 1555,
 ('NEW', 'FIN', 'RELEASE', 'CODE NOK', 'BILLED'): 977,
 ('NEW',
  'CHANGE DIAGN',
  'FIN',
  'RELEASE',
  'CODE OK',
  'REOPEN',
  'FIN',
  'RELEASE',
  'CODE OK',
  'BILLED'): 869,
 ('NEW', 'FIN', 'RELEASE', 'CODE OK', 'BILLED', 'CODE OK'): 512,
 ('NEW', 'FIN', 'RELEASE', 'CODE NOK', 'EMPTY'): 400,
 ('NEW', 'FIN', 'RELEASE', 'BILLED'): 359,
 ('NEW',
  'CHANGE DIAGN',
  'FIN',
  'RELEASE',
  'CODE OK',
  'BILLED',
  'STORNO',
  'REJECT',
  'BILLED'): 314,
 ('NEW',
  'FIN',
  'RELEASE',
  'CODE OK',
  'REOPEN',
  'FIN',
  'RELEASE',
  'CODE OK',
  'BILLED'): 273,
 ('NEW',
  'CHANGE DIAGN',
  'FIN',
  'RELEASE',
  'CODE OK',
  'BILLED',
  'STORNO',
  'REJECT

Distance Computation:

Clustering Algorithm:

Evaluation:

Interpretation and Visualization: